In [ ]:
#pip install langchain, openai, tiktoken (for tokenization of words), qdrant clieant, pyPDF2

In [30]:
import os
os.environ['OPENAI_API_KEY']="---"
os.environ['QDRANT_HOST'] = "---"
os.environ['QDRANT_API_KEY'] = "---"
openai.api_key = "---"

In [47]:
import openai
response = openai.Embedding.create(
    input="My name is Adriraj",
    model="text-embedding-ada-002"
)
embeddings = response['data'][0]['embedding']
embeddings[:5]


[-0.00930609367787838,
 0.0033837351948022842,
 -0.013574400916695595,
 -0.009516549296677113,
 -0.00981250312179327]

In [3]:
# Create Qdrant Client
from qdrant_client import QdrantClient

client = QdrantClient(
    url="---", 
    api_key="---",
)

In [4]:
# Create collection/ database using client
from qdrant_client.http import models

client.recreate_collection(
    collection_name="my-collection",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)

True

In [5]:
# Create vector store
from langchain.vectorstores import Qdrant
doc_store = Qdrant(
    client=client,
    collection_name="my-collection",
    embeddings=embeddings
)

/Users/adriraj/Library/Python/3.9/lib/python/site-packages/langchain/vectorstores/qdrant.py:141: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


In [21]:
# Retrieve text from a sample pdf
from PyPDF2 import PdfReader
pdf = "Stock_Market.pdf"
text = ""

with open(pdf,'rb') as file:
    pdf_reader = PdfReader(file)

    for page in pdf_reader.pages:
        text += page.extract_text()

In [20]:
# Split the text into chunks and put it in vector store
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
      )
chunks = text_splitter.split_text(text)

In [27]:
# Converting chunks into points which are central entity of Qdrant
import uuid
from qdrant_client.http.models import PointStruct

points = []
for idx,chunk in enumerate(chunks):
    response = openai.Embedding.create(
    input=chunk,
    model="text-embedding-ada-002" #OpenAI recommended
    )
    embeddings = response['data'][0]['embedding']
    point_id = str(uuid.uuid4())  # Generate a unique ID for the point
    points.append(PointStruct(id=point_id,payload={"text": chunk},vector=embeddings))

In [28]:
# Putting up the points on vector store
client.upsert(
    collection_name="my-collection",
    wait=True,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
# # Put vector store into retrievalChain
# from langchain.chains import RetrievalQA
# from langchain.llms import OpenAI

# qa = RetrievalQA.from_chain_type(
#     llm = OpenAI(),
#     chain_type = "stuff",
#     retriever = doc_store.as_retriever()
# )

In [51]:
query = "What factors determine interest rates"
response = openai.Embedding.create(
    input=query,
    model="text-embedding-ada-002"
)
embeddings = response['data'][0]['embedding']
search_result = client.search(
    collection_name="my-collection",
    query_vector=embeddings,
    limit=3
)
prompt=""
for result in search_result:
    prompt += result.payload['text']
concatenated_string = " ".join([prompt,query])
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": concatenated_string}
    ]
)
completion.choices[0].message.content

'The factors that determine interest rates include:\n- The demand for money: When there is high demand for borrowing, interest rates tend to increase. Conversely, when there is low demand, interest rates tend to decrease.\n- The level of government borrowings: Government borrowing can affect interest rates as it increases the demand for funds and can compete with private borrowers.\n- The supply of money: When the money supply increases, it can lead to lower interest rates as there is more money available for lending. Conversely, a decrease in the money supply can lead to higher interest rates.\n- Inflation rate: Inflation erodes the purchasing power of money, so lenders demand higher interest rates to compensate for the expected loss in value over time.\n- Policies of the central bank (such as the Reserve Bank of India) and government: These policies can influence interest rates through measures such as setting benchmark interest rates, implementing monetary policy, and regulating the

In [50]:
search_result

[ScoredPoint(id='663af104-10d2-4756-8c28-0635a4c39c20', version=0, score=0.71125096, payload={'text': 'national exchanges, which are permitted to have nationwide trading since inception. NSE was incorporated as a national stock exchange.\n/xrhombus What is an ‘Equity’/Share?\n Total equity capital of a company is divided into equal units of small \ndenominations, each called a share. For example, in a company the total equity capital of Rs 2,00,00,000 is divided into 20,00,000 units of Rs 10 each. Each such unit of Rs 10 is called a Share. Thus, the company then is 11 said to have 20,00,000 equity shares of Rs 10 each. The holders of such shares are members of the company and have voting rights.\n/xrhombus What is a ‘Debt Instrument’?\n Debt instrument represents a contract whereby one party lends \nmoney to another on pre-determined terms with regards to rate and periodicity of interest, repayment of principal amount by the borrower 7Basics of Financial Markets\nto the lender. In the 